# Optical Flow
## Lucas-Kanade algorithm (tracks certain points in frame)
1. [CV] takes in a given set of points and a frame
2. [CV] attempts to find the given points in the following frame

## Gunner Farneback algorithm (tracks all points within frame)
1. [CV] calculates dense optical flow (all points in frame)
2. [CV] color non-moving objects black and assign different colors to moving objects

In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# using the first frame of the video: detect 10 corners -> track them
corner_tracking_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

# Lucas-K parameters
lk_params = dict(winSize = (200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [22]:
cap = cv2.VideoCapture(0)
ret, prev_frame = cap.read()
prev_frame = cv2.flip(prev_frame, 1)

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Tracking points : top 10 corners
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_tracking_params)

# drawing lines mask
mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)

    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]

    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()

        cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (255,255,255), 3)

        frame = cv2.circle(frame,(int(x_new),int(y_new)),8,(255,255,255),-1)

    img = cv2.add(frame,mask)
    cv2.imshow('Tracking', img)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cap.release()
cv2.destroyAllWindows()

In [18]:
cap.release()
cv2.destroyAllWindows()